In [ ]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt

# Redes Convolucionales

[Slides 49-88](https://docs.google.com/presentation/d/1IJ2n8X4w8pvzNLmpJB-ms6-GDHWthfsJTFuyUqHfXg8/edit#slide=id.g3a1a71fe7e_8_192)

# Red Convolucional en PyTorch

Las redes neuronales convolucionales utilizan principalmente tres tipos de capas

## [Capas convolucionales](https://pytorch.org/docs/stable/nn.html#convolution-layers)

- Las neuronas de estas capas se organizan en filtros 
- Se realiza la correlación cruzada entre la imagen de entrada y los filtros
- Existen capas convolucionales 1D, 2D y 3D


[Visualización de convoluciones con distintos tamaños, strides, paddings, dilations](https://github.com/vdumoulin/conv_arithmetic)

Los argumentos de la capa convolución de dos dimensiones son:

```python
torch.nn.Conv2d(in_channels, #Cantidad de canales de la imagen de entrada
                out_channels, #Cantidad de bancos de filtro
                kernel_size, #Tamaño de los filtros (entero o tupla)
                stride=1, #Paso de los filtros
                padding=0, #Cantidad de filas y columnas para agregar a la entrada antes de filtrar
                dilation=1, #Espacio entre los pixeles de los filtros
                groups=1, #Configuración cruzada entre filtros de entrada y salida
                bias=True,  #Utilizar sesgo (b)
                padding_mode='zeros' #Especifica como agregar nuevas filas/columnas (ver padding)
                )
```

## [Capas de pooling](https://pytorch.org/docs/stable/nn.html#pooling-layers)

- Capa que reduce la dimensión (tamaño) de su entrada
- Se usa tipicamente luego de una capa de convolución "activada"
- Realiza una operación no entrenable: 
    - Promedio de los píxeles en una región (kernel_size=2, stride=2)
    
            1 2 1 0
            2 3 1 2      2.00 1.00
            0 1 0 1      0.75 0.25
            2 0 0 0
            
    - Máximo de los pixeles en una región (kernel_size=2, stride=2)
   
            1 2 1 0
            2 3 1 2      3 2
            0 1 0 1      2 1
            2 0 0 0
- Estas capas ayudan a reducir la complejidad del modelo
- También otorgan "invarianza local a la traslación", es decir que la posición donde estaba el patrón es menos relevante luego de aplicar pooling

Los argumentos de MaxPooling para entradas de dos dimensiones son:

```python
torch.nn.MaxPool2d(kernel_size, # Mismo significado que en Conv2d
                   stride=None, # Mismo significado que en Conv2d
                   padding=0, #Mismo significado que en Conv2d
                   dilation=1, #Mismo significado que en Conv2d
                   return_indices=False, #Solo necesario para hacer unpooling
                   ceil_mode=False #Usar ceil en lugar de floor para calcular el tamaño de la salida
                  )
```

## [Capas completamente conectadas](https://pytorch.org/docs/stable/nn.html#torch.nn.Linear)

- Idénticas a las usadas en redes tipo MLP
- Realizan la operación: $Z = WX + b$

Los argumentos son:

```python
torch.nn.Linear(in_features, #Neuronas en la entrada
                out_features,  #Neuronas en la salida
                bias=True  #Utilizar sesgo (b)
                )
```

# [Torchvision](https://pytorch.org/docs/stable/torchvision/index.html)

Es una librería utilitaria de PyTorch que facilita considerablemente el trabajo con imágenes

- Funcionalidad para descargar sets de benchmark: MNIST, CIFAR, IMAGENET, ...
- Modelos clásicos pre-entrenados: AlexNet, VGG, GoogLeNet, ResNet
- Funciones para importar imágenes en distintos formatos
- Funciones de transformación para hacer aumentación de datos en imágenes

## Ejemplo: Base de datos de imágenes de dígitos manuscritos MNIST

- Imágenes de 28x28 píxeles en escala de grises
- Diez categorías: Dígitos manuscritos del cero al nueve
- 60.000 imágenes de entrenamiento, 10.000 imágenes de prueba
- Por defecto las imágenes vienen en [formato PIL](https://pillow.readthedocs.io/en/stable/) (entero 8bit), usamos la transformación [`ToTensor()`](https://pytorch.org/docs/stable/torchvision/transforms.html#torchvision.transforms.ToTensor) para convertirla a tensor en float32

In [ ]:
import torchvision

mnist_train_data = torchvision.datasets.MNIST(root='~/datasets/',
                                              train=True, download=True, 
                                              transform=torchvision.transforms.ToTensor())

mnist_test_data = torchvision.datasets.MNIST(root='~/datasets/',
                                             train=False, download=True, 
                                             transform=torchvision.transforms.ToTensor())

image, label = mnist_train_data[0]
display(len(mnist_train_data), type(image), image.dtype, type(label))
fig, ax = plt.subplots(1, 10, figsize=(8, 1.5), tight_layout=True)
idx = np.random.permutation(len(mnist_train_data))[:10]
for k in range(10):
    image, label = mnist_train_data[idx[k]]
    ax[k].imshow(image[0, :, :].numpy(), cmap=plt.cm.Greys_r)
    ax[k].axis('off');
    ax[k].set_title(label)

## Dataloaders

Creamos dataloaders de entrenamiento y validación

In [ ]:
from torch.utils.data import Subset, DataLoader
import sklearn.model_selection

# Set de entrenamiento y validación estratíficados
sss = sklearn.model_selection.StratifiedShuffleSplit(train_size=0.75).split(mnist_train_data.data, 
                                                                            mnist_train_data.targets)
train_idx, valid_idx = next(sss)

# Data loader de entrenamiento
train_dataset = Subset(mnist_train_data, train_idx)
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=32)

# Data loader de validación
valid_dataset = Subset(mnist_train_data, valid_idx)
valid_loader = DataLoader(valid_dataset, shuffle=False, batch_size=256)

# Mi primera red convolucional para clasificar en pytorch

Clasificaremos la base de datos MNIST 

Para esto implementaremos la clásica arquitectura Lenet5

<img src="img/LeNet5.png" width="800">

La arquitectura considera
- Dos capas convolucionales con 8 y 16 bancos de filtros, respectivamente
- Las capas convolucionales usan filtros de 5x5 píxeles
- Se usa max-pooling de tamaño 2x2 y stride 2
- La primera capa convolucional espera un minibatch de imágenes de 1 canal (blanco y negro)
- Usaremos la función de activación [Rectified Linear Unit (ReLU)](https://pytorch.org/docs/stable/nn.html#relu)
- Se usan tres capas completamente conectadas con 120, 84 y 10 neuronas, respectivamente

> Podemos usar `reshape` o `view` para convertir un tensor de 4 dimensiones a dos dimensiones.  Esto prepara un tensor que sale de una capa convolucional (o pooling) para ingresarlo a las capas completamente conectadas

In [ ]:
import torch

class Lenet5(torch.nn.Module):
    
    def __init__(self):
        super(type(self), self).__init__()
        pass
        
    def forward(self, x):
        pass
    
model = Lenet5()
display(model)
model.forward(mnist_train_data[0][0].unsqueeze(0))

### Clasificación multiclase en PyTorch

Para hacer clasificación con **más de dos categorías** usamos la [entropía cruzada](https://pytorch.org/docs/stable/nn.html#torch.nn.CrossEntropyLoss)

```python
    torch.nn.CrossEntropyLoss()
```

- Si el problema de clasificación es de $M$ categorías la última capa de la red debe tener $M$ neuronas
- Adicionalmente no se debe usar función de activación ya que `CrossEntropyLoss` la aplica de forma interna

Para evaluar la red debemos aplicar de forma manual 
- `torch.nn.Softmax(dim=1)`
- `torch.nn.LogSoftmax(dim=1)`

a la salida de la red

> Luego podemos usar el atributo `argmax(dim=1)` para encontrar la clase más probable

### Gradiente descendente con paso adaptivo

Para acelerar el entrenamiento podemos usar un algoritmo de [gradiente descendente con paso adaptivo](https://arxiv.org/abs/1609.04747)

Un ejemplo ampliamente usado es [Adam](https://arxiv.org/abs/1412.6980)

- Se utiliza la historia de los gradientes
- Se utiliza momentum (inercia)
- Cada parámetro tiene un paso distinto

```python
    torch.optim.Adam(params,  #Parámetros de la red neuronal
                     lr=0.001,  #Tasa de aprendizaje inicial
                     betas=(0.9, 0.999),  #Factores de olvido de los gradientes históricos
                     eps=1e-08, #Término para evitar división por cero
                     weight_decay=0, #Regulariza los pesos de la red si es mayor que cero
                     amsgrad=False #Corrección para mejorar la convergencia de Adam en ciertos casos
                     )
```

**Atención**

Esta es un área de investigación activa. [Papers recientes indican que Adam llega a un óptimo más rápido que SGD, pero ese óptimo podría no ser mejor que el obtenido por SGD](https://arxiv.org/abs/1712.07628)

> Siempre prueba tus redes con distintos optimizadores


# Entrenamiento de la red convolucional

- Si tenemos acceso a una GPU podemos usar el atributo `.cuda()` o `.to()` para enviar el modelo y los datos a la GPU para acelerar los cálculos
- Actualizamos los parámetros en el conjunto de entrenamiento
- Medimos la convergencia en el conjunto de validación
- Guardamos el modelo con mejor error de validación
- Usaremos `ignite` y `Tensorboard` para entrenar. Guardaremos los entrenamientos en `/tmp/tensorboard/`

In [ ]:
from ignite.engine import Engine, Events
from ignite.metrics import Loss, Accuracy

model = Lenet5()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = torch.nn.CrossEntropyLoss(reduction='sum')
max_epochs = 100  
device = torch.device('cpu')
#device = torch.device('cuda:0')

model = model.to(device)

# Esto es lo que hace el engine de entrenamiento
def train_one_step(engine, batch):
    optimizer.zero_grad()
    x, y = batch
    x, y = x.to(device), y.to(device)
    yhat = model.forward(x)
    loss = criterion(yhat, y)
    loss.backward()
    optimizer.step()
    return loss.item() # Este output puede llamar luego como trainer.state.output

# Esto es lo que hace el engine de evaluación
def evaluate_one_step(engine, batch):
    with torch.no_grad():
        x, y = batch
        x, y = x.to(device), y.to(device)
        yhat = model.forward(x)
        #loss = criterion(yhat, y)
        return yhat, y

trainer = Engine(train_one_step)
evaluator = Engine(evaluate_one_step)
metrics = {'Loss': Loss(criterion), 'Acc': Accuracy()}
for name, metric in metrics.items():
    metric.attach(evaluator, name)

In [ ]:
import time
from torch.utils.tensorboard import SummaryWriter
from ignite.handlers import ModelCheckpoint

# Contexto de escritura de datos para tensorboard
with SummaryWriter(log_dir=f'/tmp/tensorboard/run{time.time_ns()}') as writer:

    @trainer.on(Events.EPOCH_COMPLETED(every=1)) # Cada 1 epocas
    def log_results(engine):
        # Evaluo el conjunto de entrenamiento
        evaluator.run(train_loader) 
        writer.add_scalar("train/loss", evaluator.state.metrics['Loss'], engine.state.epoch)
        writer.add_scalar("train/accy", evaluator.state.metrics['Acc'], engine.state.epoch)
        # Evaluo el conjunto de validación
        evaluator.run(valid_loader) 
        writer.add_scalar("valid/loss", evaluator.state.metrics['Loss'], engine.state.epoch)
        writer.add_scalar("valid/accy", evaluator.state.metrics['Acc'], engine.state.epoch)
    # Guardo el mejor modelo en validación
    best_model_handler = ModelCheckpoint(dirname='.', require_empty=False, filename_prefix="best", n_saved=1,
                                         score_function=lambda engine: -engine.state.metrics['Loss'],
                                         score_name="val_loss")

    # Lo siguiente se ejecuta cada ves que termine el loop de validación
    evaluator.add_event_handler(Events.COMPLETED, 
                                best_model_handler, {'lenet5': model})

    trainer.run(train_loader, max_epochs=max_epochs)

## Evaluando la red en el conjunto de test

Primero recuperamos la red con menor costo de validación

In [ ]:
model = Lenet5()
model.load_state_dict(torch.load('best_lenet5_val_loss=-0.9585.pt'))

Haremos la evaluación final del a red en el conjunto de prueba/test

Iteramos sobre el conjunto y guardamos las predicciones de la red

Con esto podemos construir una matriz de confusión y un reporte usando las herramientas de `sklearn`

In [ ]:
test_loader = DataLoader(mnist_test_data, shuffle=False, batch_size=512)

test_targets = mnist_test_data.targets.numpy()
prediction_test = []
for mbdata, label in test_loader:
    logits = model.forward(mbdata)
    prediction_test.append(logits.argmax(dim=1).detach().numpy())
prediction_test = np.concatenate(prediction_test)

from sklearn.metrics import confusion_matrix, classification_report

cm = confusion_matrix(test_targets, prediction_test)
display(cm)

print(classification_report(test_targets, prediction_test))

## Análisis de errores

Luego de evaluar la red podemos estudiar sus errores con el objeto de mejorar el modelo

Para problemas con imágenes es muy recomendable visualizar los ejemplos mal predichos por la red

Esto podría revelar
- Imágenes mal etiquetadas: Podemos cambiar su etiqueta y re-entrenar/re-evaluar
- Errores sistemáticos del modelo: Por ejemplo que siempre se equivoque con una clase u objeto en particular

Observemos algunos ejemplos mal clasificados

- Las imágenes corresponden a `digit` que no fueron predichos como `digit`
- El título de la imagen tiene la etiqueta predicha por la red

In [ ]:
digit = 8
idx = np.where((test_targets == digit) & ~(prediction_test == digit))[0]

fig, ax = plt.subplots(1, 10, figsize=(8, 1.5), tight_layout=True)
for i in range(10):
    ax[i].imshow(mnist_test_data[idx[i]][0].numpy()[0, :, :], cmap=plt.cm.Greys_r)
    ax[i].set_title(prediction_test[idx[i]])
    ax[i].axis('off')

También puede ser de interés analizar las probabilidades asignadas por la red

Para esto debemos aplicar activación Softmax a la salida

In [ ]:
image, label = mnist_test_data[idx[1]]
# Usamos unsqueeze para convertirlo en un minibatch de 1 elemento:
y = torch.nn.Softmax(dim=1)(model.forward(image.unsqueeze(0)))

fig, ax = plt.subplots(1, 2, figsize=(5, 2), tight_layout=True)
ax[0].bar(range(10), height=y.detach().numpy()[0])
ax[0].set_xticks(range(10))
ax[1].set_title("Etiqueta: %d" %(label))
ax[1].imshow(image.numpy()[0, :, :], cmap=plt.cm.Greys_r);
ax[1].axis('off');

### Visualizando los filtros aprendidos

In [ ]:
w = model.conv1.weight.data.numpy()
M = w.shape[0]
fig, ax = plt.subplots(1, M, figsize=(7, 2), tight_layout=True)

for i in range(M):    
    ax[i].imshow(w[i, 0, :, :])
    ax[i].axis('off')